#Increasing under-represented samples and standardizing it and the updating metadata(creating a new df)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Use filtered metadata file to find under-represented class and create a metadata_df

In [ ]:
import pandas as pd

metadata_df = pd.read_csv('/content/drive/MyDrive/DL project/labels/filtered_label.csv')

def find_under_represented_classes(metadata_df, class_label_column='class_label', threshold=0.1):
  """
  Finds under-represented class labels in metadata.

  Args:
    metadata_df: pandas DataFrame containing metadata.
    class_label_column: Name of the column containing class labels.
    threshold: The minimum percentage of samples a class should have to be considered
      well-represented. Defaults to 0.1 (10%).

  Returns:
    A list of under-represented class labels.
  """
  class_counts = metadata_df['Condition'].value_counts(normalize=True)
  under_represented_classes = class_counts[class_counts < threshold].index.tolist()
  return under_represented_classes

# Example usage:

under_represented_classes = find_under_represented_classes(metadata_df)
print(f"Under-represented classes: {under_represented_classes}")

Under-represented classes: ['asthma', 'pneumonia', 'urti', 'bronchiectasis', 'bronchiolitis', 'lung fibrosis', 'bronchitis', 'plueral effusion']


In [ ]:
def calculate_over_representation(metadata_df, class_label_column='class_label'):
  """
  Calculates the over-representation factor for each class.

  Args:
    metadata_df: pandas DataFrame containing metadata.
    class_label_column: Name of the column containing class labels.

  Returns:
    A pandas Series containing the over-representation factor for each class.
  """
  class_counts = metadata_df['Condition'].value_counts(normalize=True)
  mean_representation = class_counts.mean()
  over_representation_factor = class_counts / mean_representation
  return over_representation_factor

# Example usage:
over_representation = calculate_over_representation(metadata_df)
print(over_representation)

Condition
copd                6.902357
healthy             1.178451
asthma              0.816498
pneumonia           0.437710
urti                0.193603
bronchiectasis      0.134680
bronchiolitis       0.109428
lung fibrosis       0.101010
bronchitis          0.075758
plueral effusion    0.050505
Name: proportion, dtype: float64


# Segment audio files of under-represented classes:

Identify name of underrepresented classes' audio files to segment from metadata_df

In [ ]:
def get_audio_filenames_for_under_represented_classes(metadata_df, under_represented_classes):
  """
  Gets a list of audio filenames corresponding to under-represented classes.

  Args:
    metadata_df: pandas DataFrame containing metadata.
    under_represented_classes: A list of under-represented class labels.

  Returns:
    A list of audio filenames.
  """
  audio_filenames = metadata_df[metadata_df['Condition'].isin(under_represented_classes)]['Audio File Name'].tolist()
  return audio_filenames

# Example usage:
audio_filenames_to_segment = get_audio_filenames_for_under_represented_classes(metadata_df, under_represented_classes)
print(f"Audio filenames to segment: {audio_filenames_to_segment}")

Audio filenames to segment: ['105_1b1', '103_2b2', '101_1b1', '116_1b2', '119_1b1', '122_2b1', '122_2b2', '122_2b3', '122_2b2', '122_2b2', '122_2b1', '122_2b1', '122_2b3', '131_1b1', '135_2b3', '135_2b1', '135_2b2', '135_2b3', '135_2b2', '135_2b1', '137_1b1', '140_2b3', '149_1b1', '148_1b1', '149_1b1', '161_1b1', '169_1b1', '164_1b1', '168_1b1', '173_1b1', '188_1b1', '191_2b1', '188_1b1', '190_1b1', '188_1b1', '197_1b1', '196_1b1', '197_1b1', '201_1b2', '201_1b1', '201_1b1', '201_1b2', '201_1b3', '206_1b1', '206_1b1', '210_1b1', '215_1b3', '216_1b1', '215_1b2', '219_2b1', '219_2b2', '101_1b1', '226_1b1', '226_1b1', '111_1b3', '111_1b2', '122_2b3', '116_1b2', '129_1b1', '135_2b2', '135_2b1', '135_2b3', '135_2b1', '135_2b3', '137_1b1', '135_2b3', '135_2b2', '140_2b2', '140_2b2', '140_2b3', '150_1b2', '149_1b1', '161_1b1', '165_1b1', '165_1b1', '167_1b1', '165_1b1', '167_1b1', '169_1b2', '188_1b1', '191_2b1', '191_2b2', '201_1b3', '206_1b1', '210_1b1', '216_1b1', '219_2b2', '219_2b1', '21

In [ ]:
len(audio_filenames_to_segment)

228

> Identify audio files names from list of audio_files_names to be segmented using partial match.

1.   Performed 6 second segmentation of identified audio files and saved (Sendmented_audio files)
2.   created new metadata with new segmented files named segmentwise(updated_metadata)


In [ ]:
import librosa
import soundfile as sf
import os
import glob
import pandas as pd

def segment_audio_and_update_metadata(audio_dir, audio_filenames_to_segment, metadata_df, output_csv_path, output_audio_dir, segment_duration=6):
  """
  Segments audio files and updates metadata.

  Args:
    audio_dir: Directory containing the original audio files.
    audio_filenames_to_segment: List of audio filenames to segment.
    metadata_df: Original metadata DataFrame.
    output_csv_path: Path to save the updated metadata CSV.
    output_audio_dir: Directory to save the segmented audio files.
    segment_duration: Duration of each segment in seconds. Defaults to 6.
  """
  new_metadata = []

  for metadata_filename in audio_filenames_to_segment:
    # Find the actual audio file using partial match
    audio_files = glob.glob(os.path.join(audio_dir, metadata_filename + '*.wav'))

    if not audio_files:
      print(f"Warning: No audio file found for {metadata_filename}")
      continue

    # Extract filename without extension
    full_filename = audio_files[0]
    filename_without_ext = os.path.splitext(os.path.basename(full_filename))[0]

    try:
      # Load the audio file
      y, sr = librosa.load(full_filename)

      # Calculate the number of segments
      num_segments = int(librosa.get_duration(y=y, sr=sr) // segment_duration)

      # If audio is shorter than segment_duration, skip segmentation
      if num_segments == 0:
        continue

      # Segment the audio
      for i in range(num_segments):
        start_time = i * segment_duration
        end_time = (i + 1) * segment_duration
        segment = y[int(start_time * sr):int(end_time * sr)]

        # Create a new identifier name for the segment
        segment_filename = f"{filename_without_ext}_{i+1}"

        # Save the segment to a file
        sf.write(os.path.join(output_audio_dir, segment_filename + '.wav'), segment, sr)

        # Update metadata for the segment
        original_row = metadata_df[metadata_df['Audio File Name'] == metadata_filename].iloc[0]  # Assuming 'Audio File Name' is the column name
        new_row = original_row.copy()
        new_row['Audio File Name'] = segment_filename
        new_metadata.append(new_row.to_dict())

    except Exception as e:
      print(f"Error processing {filename_without_ext}: {e}")

  # Create a new DataFrame from the updated metadata
  new_metadata_df = pd.DataFrame(new_metadata)

  # Save the updated metadata to a CSV file
  new_metadata_df.to_csv(output_csv_path, index=False)

# Example usage:
audio_dir = '/content/drive/MyDrive/DL project/Merged and preprocessed audio/Merged audio files'
output_csv_path = '/content/drive/MyDrive/DL project/labels/updated_metadata.csv'
output_audio_dir = '/content/drive/MyDrive/DL project/Merged and preprocessed audio/Segmenting_longaudio'
segment_audio_and_update_metadata(audio_dir, audio_filenames_to_segment, metadata_df, output_csv_path, output_audio_dir, segment_duration=6)

Check duration of audio in segmented audio files


In [ ]:
import librosa
import os

def check_audio_durations(audio_dir):
  """Checks and prints the duration of audio files in a directory.

  Args:
    audio_dir: Directory containing the audio files.
  """
  for filename in os.listdir(audio_dir):
    if filename.endswith(".wav"):
      filepath = os.path.join(audio_dir, filename)
      duration = librosa.get_duration(filename=filepath)
      print(f"Duration of {filename}: {duration:.2f} seconds")

# Example usage:
audio_dir = '/content/drive/MyDrive/DL project/Merged and preprocessed audio/Segmenting_longaudio'  # Replace with your directory
check_audio_durations(audio_dir)

<ipython-input-19-ad4fd9171f0c>:13: FutureWarning: get_duration() keyword argument 'filename' has been renamed to 'path' in version 0.10.0.
	This alias will be removed in version 1.0.
  duration = librosa.get_duration(filename=filepath)


Duration of 101_1b1_Pr_sc_Meditron_1.wav: 6.00 seconds
Duration of 101_1b1_Pr_sc_Meditron_2.wav: 6.00 seconds
Duration of 101_1b1_Pr_sc_Meditron_3.wav: 6.00 seconds
Duration of 116_1b2_Pl_sc_Meditron_1.wav: 6.00 seconds
Duration of 116_1b2_Pl_sc_Meditron_2.wav: 6.00 seconds
Duration of 116_1b2_Pl_sc_Meditron_3.wav: 6.00 seconds
Duration of 119_1b1_Ar_sc_Meditron_1.wav: 6.00 seconds
Duration of 119_1b1_Ar_sc_Meditron_2.wav: 6.00 seconds
Duration of 119_1b1_Ar_sc_Meditron_3.wav: 6.00 seconds
Duration of 122_2b1_Al_mc_LittC2SE_1.wav: 6.00 seconds
Duration of 122_2b1_Al_mc_LittC2SE_2.wav: 6.00 seconds
Duration of 122_2b1_Al_mc_LittC2SE_3.wav: 6.00 seconds
Duration of 122_2b2_Ar_mc_LittC2SE_1.wav: 6.00 seconds
Duration of 122_2b2_Ar_mc_LittC2SE_2.wav: 6.00 seconds
Duration of 122_2b2_Ar_mc_LittC2SE_3.wav: 6.00 seconds
Duration of 122_2b3_Tc_mc_LittC2SE_1.wav: 6.00 seconds
Duration of 122_2b3_Tc_mc_LittC2SE_2.wav: 6.00 seconds
Duration of 122_2b3_Tc_mc_LittC2SE_3.wav: 6.00 seconds
Duration o